In [ ]:
%cd "full/path/to/codebase"
from pathlib import Path

import polars as pl
from qdrant_client.models import Prefetch
from tqdm import tqdm

from src.data.dataset import LawCorpus, TestDataset
from src.db.qdrant import Qdrant
from src.nn.text_embedding import TextEmbedding

/home/anhvd27/miniforge3/envs/mlqa-tsr/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/home/anhvd27/miniforge3/envs/mlqa-tsr/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/home/anhvd27/projects/vlsp/mlqa-tsr/codebase


/home/anhvd27/miniforge3/envs/mlqa-tsr/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
text_embedding_model = TextEmbedding()

flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn i

In [ ]:
qdrant = Qdrant()

# path to private test folder, perhaps one need to slightly change the file names and image path
# read the code in src/data/dataset.py
test_set_1 = TestDataset(
    root_dir=Path("../data/private_test"), law_corpus=LawCorpus(Path("../data/law_db")), task_number=1
)
image_feature_df = pl.read_json("assets/private_test_image_feature.json")

In [ ]:
collection_name = "train_data"
for i in tqdm(range(len(test_set_1))):
    question = test_set_1[i]["question"]
    image_name = Path(test_set_1[i]["image_path"]).name
    image_feature_point = image_feature_df.filter(pl.col("image_name") == image_name).row(0, named=True)

    image_object_feature_list_vector = [[0.0] * 2304]
    if image_feature_point["object_feature_list"]:
        image_object_feature_list_vector = [
            img_obj["object_feature"] for img_obj in image_feature_point["object_feature_list"]
        ]

    results = qdrant.client.query_points(
        collection_name=collection_name,
        prefetch=Prefetch(
            prefetch=Prefetch(
                query=text_embedding_model.infer_single(question),
                using="text_vector",
                limit=10,
            ),
            query=image_feature_point["general_feature"],
            using="image_general_feature_vector",
            limit=5,
        ),
        query=image_object_feature_list_vector,
        using="image_object_feature_list_vector",
        limit=3,
        with_payload=True,
    )
    fmt_relevant_articles = []
    for result_point in results.points:
        if result_point.payload:
            for ra in result_point.payload["relevant_articles"]:
                fmt_relevant_articles.append(
                    {
                        "law_id": ra["law_id"],
                        "article_id": ra["article_id"],
                    },
                )
    if len(results.points) > 1:
        fmt_relevant_articles = list(set(tuple(sorted(d.items())) for d in fmt_relevant_articles))
        fmt_relevant_articles = [dict(t) for t in fmt_relevant_articles]
    test_set_1.index_result(index=i, relevant_articles=fmt_relevant_articles)

100%|██████████| 146/146 [00:03<00:00, 36.98it/s]


In [5]:
test_set_1.save_results("assets/private_submissions/submission_task1.json")

True